In [ ]:
# Cài gói cần thiết (nếu chạy local, bỏ dấu # ở dòng dưới)
#!pip install -U scikit-learn

# Trên môi trường JupyterLite/Pyodide dùng piplite để cài gói Python
import piplite
# Cài các thư viện dùng trong bài: thanh tiến độ, vẽ, xử lý dữ liệu, số học, học máy
await piplite.install(['tqdm', 'seaborn', 'pandas', 'numpy', 'scikit-learn'])


In [ ]:
# Thư viện hiển thị tiến trình
from tqdm import tqdm

# Số học, xử lý dữ liệu
import numpy as np
import pandas as pd
from itertools import accumulate  # hiện chưa dùng, có thể bỏ nếu cần

# Vẽ biểu đồ/khám phá dữ liệu
import matplotlib.pyplot as plt
import seaborn as sns
# Hiển thị biểu đồ trực tiếp trong notebook
%matplotlib inline

# Tiền xử lý và thống kê
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression  # nhập nhưng không dùng trong mô hình cuối
from sklearn.datasets import load_digits, load_wine  # không dùng ở bài này
from scipy.stats import boxcox
from scipy.stats.mstats import normaltest

# Hồi quy tuyến tính và đánh giá mô hình
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score 
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

In [ ]:
# Kiểm tra phiên bản Scikit-Learn để đảm bảo tương thích
import sklearn; print("Scikit-Learn", sklearn.__version__)

In [ ]:
# Tắt cảnh báo để output gọn hơn (lưu ý: không nên dùng khi cần debug)
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
# Tải dữ liệu trong môi trường Pyodide bằng pyfetch (bất đồng bộ)
from pyodide.http import pyfetch
 
async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())
 
# Đường dẫn tệp dữ liệu trên cloud
path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML240EN-SkillsNetwork/labs/data/CarPrice_Assignment.csv"
 
# Nếu chạy local và đã có file, có thể bỏ qua dòng dưới
await download(path, "CarPrice_Assignment.csv")

In [ ]:
# Đọc file CSV đã tải vào DataFrame
import pandas as pd

data = pd.read_csv("CarPrice_Assignment.csv")

# Xem nhanh 5 dòng đầu để nắm cấu trúc dữ liệu
print("The first 5 rows of the dataframe")
data.head(5)

In [ ]:
# Thông tin kiểu dữ liệu, số lượng non-null của từng cột
data.info()

In [ ]:
# Kiểm tra tổng số giá trị thiếu theo từng cột
data.isnull().sum()

In [ ]:
# Kiểm tra trùng lặp theo khóa 'car_ID' (True nghĩa là không có bản ghi trùng)
sum(data.duplicated(subset = 'car_ID')) == 0

In [ ]:
# Liệt kê các giá trị hãng xe trong cột tên để chuẩn hóa sau này
data["CarName"].unique()

In [ ]:
# Tách hãng xe (brand) từ cột tên xe: lấy từ đầu tiên và chuyển thành chữ thường
data['brand'] = data.CarName.str.split(' ').str.get(0).str.lower()

In [ ]:
# Kiểm tra danh sách hãng sau khi tách để xem có lỗi chính tả
data.brand.unique()

In [ ]:
# Chuẩn hóa lỗi chính tả/biến thể tên hãng để gộp đúng thương hiệu
data['brand'] = data['brand'].replace(['vw', 'vokswagen'], 'volkswagen')
data['brand'] = data['brand'].replace(['maxda'], 'mazda')
data['brand'] = data['brand'].replace(['porcshce'], 'porsche')
data['brand'] = data['brand'].replace(['toyouta'], 'toyota')

In [ ]:
# Kiểm tra lại danh sách hãng sau chuẩn hóa
data.brand.unique()

In [ ]:
# Biểu đồ tần suất theo hãng xe để xem phân bố nhãn danh mục
fig, ax = plt.subplots(figsize = (15,5))
plt1 = sns.countplot(x=data['brand'], order=pd.value_counts(data['brand']).index)
plt1.set(xlabel = 'Brand', ylabel= 'Count of Cars')
plt.show()
plt.tight_layout()

In [ ]:
# Loại bỏ cột không cần thiết cho mô hình
# - 'car_ID': chỉ là định danh
# - 'symboling': điểm số liên quan rủi ro bảo hiểm, không dùng ở đây
# - 'CarName': đã tách brand nên không cần
data.drop(['car_ID', 'symboling', 'CarName'],axis = 1, inplace = True)

In [ ]:
# Thăm dò nhanh một vài biến danh mục để xác định giá trị hợp lệ
data.fueltype.unique()
data["enginelocation"].value_counts()

In [ ]:
# Tính giá trung bình theo hãng để phân nhóm phân khúc thương hiệu
data_comp_avg_price = data[['brand','price']].groupby('brand', as_index = False).mean().rename(columns={'price':'brand_avg_price'})


In [ ]:
# Gộp giá trung bình theo hãng vào bộ dữ liệu chính
data = data.merge(data_comp_avg_price, on = 'brand')

In [ ]:
# Thống kê mô tả cho cột giá trung bình theo hãng
data.brand_avg_price.describe()

In [ ]:
# Phân loại hãng theo mức giá trung bình: Budget / Mid_Range / Luxury
data['brand_category'] = data['brand_avg_price'].apply(lambda x : "Budget" if x < 10000 
                                                     else ("Mid_Range" if 10000 <= x < 20000
                                                           else "Luxury"))

In [ ]:
# So sánh phân bố giá theo các biến danh mục bằng boxplot
plt.figure(figsize=(10, 20))
plt.subplot(4,2,1)
sns.boxplot(x = 'fueltype', y = 'price', data = data)
plt.subplot(4,2,2)
sns.boxplot(x = 'aspiration', y = 'price', data = data)
plt.subplot(4,2,3)
sns.boxplot(x = 'carbody', y = 'price', data = data)
plt.subplot(4,2,4)
sns.boxplot(x = 'drivewheel', y = 'price', data = data)
plt.subplot(4,2,5)
sns.boxplot(x = 'enginetype', y = 'price', data = data)
plt.subplot(4,2,6)
sns.boxplot(x = 'brand_category', y = 'price', data = data)
plt.tight_layout()
plt.show()

In [ ]:
# Ma trận tương quan (chỉ numeric); sắp xếp mức độ tương quan với giá
corr_matrix = data.corr(numeric_only=True)
corr_matrix['price'].sort_values(ascending=False)

In [ ]:
# Pairplot để quan sát tương quan cặp và phân phối mỗi biến
sns.pairplot(data)
plt.show()

In [ ]:
# Quan hệ tuyến tính gần đúng giữa dung tích động cơ/horsepower và giá
fig, (ax1, ax2) = plt.subplots(figsize = (12,8), ncols=2,sharey=False)
sns.scatterplot( x = data.enginesize, y = data.price,  ax=ax1)
sns.regplot(x=data.enginesize, y=data.price, ax=ax1)
 
sns.scatterplot(x = data.horsepower,y = data.price, ax=ax2)
sns.regplot(x=data.horsepower, y=data.price, ax=ax2);


In [ ]:
# Cân nặng xe cũng có xu hướng tương quan với giá
sns.regplot(x=data.curbweight, y=data.price, data=data)

In [ ]:
# Residual plot để kiểm tra giả định tuyến tính/độ phân tán phần dư
plt.subplots(figsize = (12,8))
sns.residplot(x=data["enginesize"], y=data["price"])

In [ ]:
# Hàm vẽ 3 đồ thị (histogram, QQ-plot, boxplot) để kiểm tra phân phối một biến
def plotting_3_chart(data, feature):
    import seaborn as sns
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    from scipy import stats
    import matplotlib.style as style
    style.use('fivethirtyeight')

    fig = plt.figure(constrained_layout=True, figsize=(12,8))
    grid = gridspec.GridSpec(ncols=3, nrows=3, figure=fig)

    # Histogram
    ax1 = fig.add_subplot(grid[0, :2])
    ax1.set_title('Histogram')
    sns.distplot(data.loc[:,feature], norm_hist=True, ax = ax1)

    # QQ-plot
    ax2 = fig.add_subplot(grid[1, :2])
    ax2.set_title('QQ_plot')
    stats.probplot(data.loc[:,feature], plot = ax2)

    # Boxplot
    ax3 = fig.add_subplot(grid[:, 2])
    ax3.set_title('Box Plot')
    sns.boxplot(data.loc[:,feature], orient='v', ax = ax3);
    
plotting_3_chart(data, 'price')

In [ ]:
# Lưu bản sao dữ liệu trước biến đổi để so sánh sau này
previous_data = data.copy()

In [ ]:
# Kiểm định độ chuẩn (D’Agostino-Pearson) cho biến giá hiện tại
normaltest(data.price.values)

In [ ]:
# Biến đổi log để làm phân phối giá gần chuẩn hơn (giảm lệch phải)
data['price'] = np.log(data['price'])
plotting_3_chart(data, 'price')

In [ ]:
# Kiểm định lại sau khi log-transform
normaltest(data.price.values)

In [ ]:
# Thử Box-Cox transform (yêu cầu dữ liệu dương), rồi kiểm định chuẩn
cp_result = boxcox(previous_data.price)
boxcox_price = cp_result[0]

normaltest(boxcox_price)

In [ ]:
# Heatmap tương quan tổng thể giữa các biến
plt.figure(figsize = (30, 25))
sns.heatmap(data.corr(), annot = True, cmap="YlGnBu")
plt.show()

In [ ]:
# Chọn tập biến đầu vào + mục tiêu 'price' để mô hình hóa
columns=['price', 'fueltype', 'aspiration','carbody', 'drivewheel','wheelbase', 'brand_category',
                  'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 'carlength','carwidth','citympg','highwaympg']

selected = data[columns]
selected.info()

In [ ]:
# Xác định các cột dạng danh mục (dtype=object)
categorical_columns = [col for col in selected.columns if selected[col].dtype == 'object']  
categorical_columns

In [ ]:
# Các cột số = (tập cột đã chọn) trừ (các cột danh mục)
numeric_columns=list(set(columns)-set(categorical_columns))
numeric_columns

In [ ]:
# Tách đặc trưng X (mọi cột trừ mục tiêu 'price')
X = selected.drop("price", axis=1)
X.head()


In [ ]:
# Biến mục tiêu y (giá đã được log-transform ở trên)
y = selected["price"].copy()
y.head()

In [ ]:
# Xem tần suất các giá trị cho từng biến danh mục (giúp kiểm tra data hiếm)
for column in  categorical_columns:
    print("column name:", column)
    print("value_count:")
    print( X[column].value_counts())

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
# Mã hóa one-hot cho biến danh mục; các cột còn lại giữ nguyên (passthrough)
one_hot = ColumnTransformer(transformers=[("one_hot", OneHotEncoder(), categorical_columns) ],remainder="passthrough")
X=one_hot.fit_transform(X)
# Sau bước này X trở thành mảng/ma trận (scipy.sparse hoặc ndarray)
type(X)

In [ ]:
# Lấy tên cột sau khi biến đổi để đặt tên DataFrame
names=one_hot.get_feature_names_out()
names

In [ ]:
# Làm gọn tên cột tạo bởi ColumnTransformer__ và OneHotEncoder_
colunm_names=[name[name.find("_")+1:] for name in  [name[name.find("__")+2:] for name in names]]
colunm_names

In [ ]:
# Tạo DataFrame từ ma trận đặc trưng đã mã hóa để dễ thao tác/xem cột
df=pd.DataFrame(data=X,columns=colunm_names)
# Có thể lưu ra CSV nếu cần
#df.to_csv('cleaned_car_data.csv', index=False)

In [ ]:
X_ = selected[categorical_columns+numeric_columns]

X_numeric=X[numeric_columns].to_numpy()
X_categorical=OneHotEncoder().fit_transform(X_[categorical_columns]).toarray()
X_=np.concatenate((X_categorical,X_numeric), axis = 1)

In [ ]:
# Hàm tạo biến giả cho một cột danh mục và nối vào DataFrame
# Lưu ý: cần thụt lề đúng để Python hiểu khối lệnh

def dummies(x, data):
    temp = pd.get_dummies(data[x], drop_first=True)
    data = pd.concat([data, temp], axis=1)
    data.drop([x], axis=1, inplace=True)
    return data

# Bản sao dữ liệu để minh họa cách tự tạo dummies (không bắt buộc vì đã có OneHotEncoder ở trên)
X_ = selected[categorical_columns + numeric_columns]

# Xem nhanh các giá trị duy nhất của từng cột danh mục
for column in categorical_columns:
    print(pd.unique(data[column]))
    # Biến đổi dummies cho từng cột (nếu muốn dùng cách thủ công)
    X_ = dummies(column, X_)

# X_ lúc này là DataFrame đã mã hóa dummies theo cách thủ công

In [ ]:
# Chia train/test để đánh giá tổng quát hóa
from sklearn.model_selection import train_test_split

In [ ]:
# Tạo tập huấn luyện/kiểm tra (30% dữ liệu làm test)
X_train, X_test, y_train, y_test = train_test_split( df, y, test_size=0.30, random_state=0)

In [ ]:
# Chuẩn hóa đặc trưng để mô hình tuyến tính ổn định hơn
from sklearn.preprocessing import StandardScaler

In [ ]:
# Khởi tạo bộ chuẩn hóa theo z-score (mean=0, std=1)
ss=StandardScaler()
ss

In [ ]:
# Fit trên train và biến đổi train; không dùng thông tin test khi fit
X_train=ss.fit_transform(X_train)

In [ ]:
# Huấn luyện mô hình hồi quy tuyến tính thường
lm = LinearRegression()
lm.fit(X_train,y_train)


In [ ]:
# Biến đổi test bằng scaler đã fit trên train, rồi dự đoán
X_test=ss.transform(X_test)
car_price_predictions = lm.predict(X_test)
car_price_predictions

In [ ]:
# Sai số bình phương trung bình trên tập test (đơn vị: log-price)
mse = mean_squared_error(y_test, car_price_predictions)
mse

In [ ]:
# R^2 trên tập test (giải thích phương sai)
lm.score(X_test,y_test)

In [ ]:
from sklearn.metrics import r2_score 

In [ ]:
# Tính R^2 bằng hàm r2_score (kết quả nên khớp với lm.score)
r2_score(y_test,car_price_predictions)

In [ ]:
# Xây dựng pipeline: chuẩn hóa -> hồi quy tuyến tính
steps=[('scaler', StandardScaler()), ('lm',  LinearRegression())]

In [ ]:
# Khởi tạo đối tượng Pipeline
pipe = Pipeline(steps=steps)

In [ ]:
# Huấn luyện pipeline trên tập train (tự động fit scaler trước)
pipe.fit(X_train,y_train)

In [ ]:
# Dự đoán và tính RMSE (đơn vị: log-price)
car_price_predictions = pipe.predict(X_test)
mse = mean_squared_error(y_test, car_price_predictions)
rmse = np.sqrt(mse)
rmse


In [ ]:
# Lưu ý: thứ tự tham số r2_score là (y_true, y_pred)
r2_score(y_test, car_price_predictions)

In [ ]:
# Pipeline end-to-end: mã hóa danh mục -> chuẩn hóa -> hồi quy
X = selected[categorical_columns+numeric_columns]
one_hot = ColumnTransformer(transformers=[("one_hot", OneHotEncoder(), categorical_columns) ],remainder="passthrough")
steps=[('one_hot',one_hot), ('scaler', StandardScaler()), ('lm',  LinearRegression())]

pipe = Pipeline(steps=steps)
pipe.fit(X,y)
car_price_predictions=pipe.predict(X)
# Sử dụng đúng thứ tự đối số: (y_true, y_pred)
r2_score(y, car_price_predictions)